In [ ]:
#Install the necessary dependencies
!sudo apt-get install swig >> /dev/null
!pip install gymnasium[box2d] >> /dev/null
!pip install gym[box2d]

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 4.1 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2


In [ ]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print("PyTorch is using the GPU:", torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print("PyTorch is not using a GPU.")
    device = torch.device('cpu')

PyTorch is using the GPU: Tesla T4


In [ ]:
print(device)

cuda


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gymnasium as gym

In [ ]:
# Set up the environment
env_name = "LunarLander-v2"

# from tf_agents.environments import suite_gym
# env = suite_gym.load(env_name)



#Default settings
# env = gym.make(
#     env_name,
#     continous = False,
#     gravity = -10.0,
#     enable_wind = False,
#     wind_power = 15.0,
#     turbulence_power = 1.5
# )

env = gym.make(
    env_name,
    continuous = False,  # We will use the LunarLander environment with discrete action space
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5,
    render_mode='rgb_array'
)






In [ ]:
# for checking if the implementation is correct
# use another environment on which REINFORCE has already been tested successful
###############################################################################

env = gym.make("CartPole-v1",render_mode="rgb_array")

In [ ]:
action_space_dim = env.action_space.n
print(f"action_space_dim:{action_space_dim}")

observation_space_dim = env.observation_space.shape[0]
print(f"observation_space_dim:{observation_space_dim}")

action_space_dim:2
observation_space_dim:4


In [ ]:
#simple REINFORCE 1

import torch
import torch.nn as nn
import torch.nn.functional as F

class PolicyNet1(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(observation_space_dim,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,32)
        self.logits_layer = nn.Linear(32,action_space_dim)
        self.ln1 = nn.LayerNorm(256)
        self.ln2 = nn.LayerNorm(128)
        self.ln3 = nn.LayerNorm(64)
        self.ln4 = nn.LayerNorm(32)

    def forward(self,x):
        x = self.fc1(x)
        x = nn.ReLU()(self.ln1(x))
        x = self.fc2(x)
        x = nn.ReLU()(self.ln2(x))
        x = self.fc3(x)
        x = nn.ReLU()(self.ln3(x))
        x = self.fc4(x)
        x = nn.ReLU()(self.ln4(x))
        x = self.logits_layer(x)
        return x


In [ ]:
# Base classes : PolicyStep & TimeStep

from typing import NamedTuple , Optional , Union

ActionType = Union[torch.Tensor,np.ndarray]
TensorOrArray = Union[torch.Tensor, np.ndarray, int, float, str, bool]

class PolicyStep(NamedTuple):
    action: ActionType

class TimeStep(NamedTuple):
    observation : TensorOrArray
    reward : TensorOrArray
    step_type : TensorOrArray
    discount : TensorOrArray

    def is_first(self):
        return np.equal(self.step_type, np.asarray(0, dtype= np.int32))
    def is_mid(self):
        return np.equal(self.step_type, np.asarray(1, dtype= np.int32))
    def is_last(self):
        return np.equal(self.step_type, np.asarray(2, dtype= np.int32))

In [ ]:
# Test 1: Correct configuration of the neural network to intake the observation from the environment

from torch.distributions.categorical import Categorical

s_0, info_0 = env.reset()

net1 = PolicyNet1()

s_0_tensor = torch.from_numpy(s_0).reshape(1,-1)

logits_0 = net1(s_0_tensor)

print("logits:",logits_0)

distribution = Categorical(logits=logits_0)

sampled_action = distribution.sample()

print("sampled_action:",sampled_action)

print(np.array(sampled_action))

logits: tensor([[-0.1234,  0.4805]], grad_fn=<AddmmBackward0>)
sampled_action: tensor([0])
[0]


In [ ]:
logits_np = np.array([[10, 0]], dtype = float)
logits_tensor = torch.from_numpy(logits_np)

distribution = Categorical(logits=logits_tensor)

sampled_action = distribution.sample()

print("sampled_action:",sampled_action)

print(np.array(sampled_action))

print(sampled_action.shape)

sampled_action: tensor([0])
[0]
torch.Size([1])


In [ ]:
# print(s_0)
# print(type(s_0))
# print(s_0.shape)

# print(logits_0)
# print(type(logits_0))
# print(logits_0.shape)

In [ ]:
#Helper functions for discounting reward & computing mean & std for baseline in REINFORCE

def discount_rewards(rewards, discount_factor):
    '''
        Converts the rewards sequence to the discounted return sequence
    '''

    discounted = np.array(rewards)
    for step in range(len(rewards)-2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(rewards,discount_factor) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    # return all_discounted_rewards
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/ reward_std for discounted_rewards in all_discounted_rewards]

In [ ]:
#function: create lists of batched obs, batched action and batched rewards

# input: nested lists of obs, action , rewards
# output: list of batched obs tensors, list of batched action tensors, list of batched rewards tensor

import random

def create_batches(obs_all, action_all, reward_all, batch_size):
    # Shuffle the data to ensure each batch is a random sample
    combined = list(zip(obs_all, action_all, reward_all))
    random.shuffle(combined)
    obs_all[:], action_all[:], reward_all[:] = zip(*combined)

    # Calculate the total number of full batches
    total_full_batches = len(obs_all) // batch_size
    if total_full_batches == 0:
        end_index = 0

    # Create batched arrays
    batched_obs = []
    batched_actions = []
    batched_rewards = []

    for i in range(total_full_batches):
        # Create a batch
        start_index = i * batch_size
        end_index = start_index + batch_size
        obs_batch = torch.from_numpy(np.concatenate(obs_all[start_index:end_index], dtype=np.float32))
        actions_batch = torch.tensor(action_all[start_index:end_index], dtype=torch.long)
        rewards_batch = torch.tensor(reward_all[start_index:end_index], dtype=torch.float32)

        # Append the batch to our list

        obs_batch=obs_batch.to(device=device)
        actions_batch=actions_batch.to(device=device)
        rewards_batch=rewards_batch.to(device=device)

        batched_obs.append(obs_batch)
        # batched_actions.append(actions_batch[:, np.newaxis])  # Add a new axis for consistency
        batched_actions.append(actions_batch) # in the `collect` method of the agent, the appended action is in shape (1,1), so actions_batch is in shape
        batched_rewards.append(rewards_batch)

    # Handle the last batch which might not be the full batch size
    if len(obs_all) % batch_size != 0:
        obs_batch = torch.from_numpy(np.concatenate(obs_all[end_index:], dtype=np.float32))
        actions_batch = torch.tensor(action_all[end_index:], dtype=torch.long)
        rewards_batch = torch.tensor(reward_all[end_index:], dtype=torch.float32)

        obs_batch=obs_batch.to(device=device)
        actions_batch=actions_batch.to(device=device)
        rewards_batch=rewards_batch.to(device=device)

        # Append the batch to our list
        batched_obs.append(obs_batch)
        # batched_actions.append(actions_batch[:, np.newaxis])
        batched_actions.append(actions_batch)
        batched_rewards.append(rewards_batch)

    # print("Created batches")
    # print(f"batch_obs shape:{batched_obs[0].shape}")
    # print(f"batch_action shape:{batched_actions[0].shape}")
    # print(f"batch_reward shape:{batched_rewards[0].shape}")

    return batched_obs, batched_actions, batched_rewards

In [ ]:
#To create a REINFORCE algorithm for the LunarLander environment, we need to have the following:

#1. A REINFORCE agent class, responsible for learning the policy through REINFORCE
#2. A REINFORCE policy class, responsible for producing the PolicyStep that interacts with the environment

class NeuralNetPolicy():
    def __init__(self,Model_Class):
        '''
        Model Class is a neural network class, that an object instantiated of this class can take the state and output an action/prob logits
        '''
        self.net = Model_Class().cuda()


    def action(self,time_step):
        action_fn = self._action
        policy_step = action_fn(time_step)

        return policy_step

    def _action(self,time_step):
        '''
        `_action` is for private use, same as all methods of which name starts with an underscore,

        here the `_action` method calls the `_distribution` method to get the logits, then it
        makes use of `tf.random.categorical` to sample an action from the given logits, finally
        it embeds the sampled action in the PolicyStep object , which is to be returned
        '''
        distribution_step = self._distribution(time_step)
        distribution = distribution_step.action

        num_samples = 1

        sampled_action = distribution.sample()

        return PolicyStep(action = sampled_action)

    def _distribution(self,time_step):
        '''
        _distribution is for private use, same as all methods of which name starts with an underscore

        here it just passes the state (which in our case is identical to the observation) to the net
        and returns the logits
        '''

        policy_logits = self.net(time_step.observation)

        distribution = Categorical(logits=policy_logits)

        return PolicyStep(action = distribution)



class REINFORCEAgent():
    def __init__(self):
        # self._policy = NeuralNetPolicy(SimplePolicyNet)
        self._policy = NeuralNetPolicy(PolicyNet1)
        self._collect_policy = self._policy

    def train(self,optimizer,obs,action,reward):
        """ method for training self.policy

        experience is a tuple consisting of 3 lists: obs, action, reward

        this `train` is designed to train on a single batch, as aligned with common ML API design
        """

        #REINFORCE learning algorithm
        #1. input the obs to the net first, and get the logits as output
        #2. set up the loss function correctly such that its gradient aligns with the policy gradient;
        #   in REINFORCE (with baseline), the gradient is proportional to the return, inversely proportional to the probability
        #   del J(theta) = (R_t - baseline)/P(A_t) * del P(A_t)
        #   or : del J(theta) = (R_t - baseline) * del ln(P(A_t))
        #   Hence, the loss function would be the cross entropy of actual action log action probabilities
        #   multiplied by (R_t - baseline)
        #3. backpropagation

        #####################################################################################################################################
        #
        # PyTorch : Calculating Cross Entropy from logits, Method 1: F.cross_entropy(logits, one_hot_label)
        #
        #####################################################################################################################################

        # logits = self._policy.net(obs)
        # action_one_hot = F.one_hot(action,num_classes=action_space_dim).to(dtype=torch.float32)

        # # both logits and action_one_hot should be of the same shape and data type : (n,d)
        # cross_entropy_vector = F.cross_entropy(logits,action_one_hot,reduction='none')

        # loss_vector = reward * cross_entropy_vector

        # loss = torch.mean(loss_vector)

        #####################################################################################################################################
        #
        # PyTorch : Calculating Cross Entropy from logits, Method 2: Categorical log_probs
        #
        # Input the batched_action as argument in log_prob, then multiply by -1 to get the Cross entropy vector
        # Multiply the CE vector with reward vector to get the batched negative objective function
        # whose negative gradients are in the same direction as gradients to increase expected return
        #
        #####################################################################################################################################

        logits = self._policy.net(obs)
        m = Categorical(logits=logits)
        CE_vector = -1 * m.log_prob(batched_action)
        loss_vector = CE_vector * batched_reward

        loss = torch.mean(loss_vector)

        # Values checking:
        # print("batched_obs", batched_obs)
        # print("logits", logits)
        # print("batched_actions", batched_actions)
        # print("action_one_hot", action_one_hot)
        # print("cross entropy vector",cross_entropy_vector)
        # print("rewards vector", batched_rewards)
        # print("loss vector",loss_vector)
        # print("loss",loss)

        # Shape checking:

        # print(f"batched obs shape,{batched_obs[j].shape}")
        # print(f"batched_actions shape, {batched_actions[j].shape}")
        # print(f"logits shape:{logits.shape}, action_one_hot shape:{action_one_hot.shape}")
        # print(f"loss vector shape:{loss_vector.shape}")
        # print(f"loss shape:{loss.shape}")

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        return loss

    def collect(self,env,n_episodes):
        all_obs_collect = []
        all_action_collect = []
        all_reward_collect = []

        for i in range(n_episodes):
            # Initialize the collection variables
            obs_collect = []
            action_collect = []
            reward_collect = []
            total_reward = 0
            done, truncate = False, False

            obs, info = env.reset()
            obs = obs.reshape(1,-1)
            obs_tensor = torch.from_numpy(obs).cuda()
            reward = np.asarray(0, dtype= np.float32)
            step_type = np.asarray(0, dtype= np.float32) # step type with content value of 0 means a FIRST step type
            discount_factor = 1

            time_step = TimeStep(obs_tensor,reward,step_type,discount_factor)
            total_reward += reward

            while not (done or truncate):
                policy_step = self.policy.action(time_step)
                action_int = policy_step.action.reshape(-1).item() # has to convert the tensor to int since the environment accepts the action as int
                action_int = int(action_int)

                #Collect experience: observation and action are collected before step()
                obs_collect.append(obs) # (obs_dim,)
                action_collect.append(action_int) # 0-dim scalar

                obs, reward, done, truncate, info = env.step(action_int)
                obs = obs.reshape(1,-1)
                obs_tensor = torch.from_numpy(obs).cuda()

                #Collect experience: reward is collected after step()
                reward_collect.append(reward) # 0-dim scalar

                if done or truncate:
                    step_type = np.asarray(2,dtype=np.int32) # step type with content value of 2 means a LAST step type
                else:
                    step_type = np.asarray(1,dtype=np.int32) # step type with content value of 1 means a MID step type

                time_step = TimeStep(obs_tensor,reward,step_type,discount_factor)
                total_reward += reward

            all_obs_collect.append(obs_collect)
            all_action_collect.append(action_collect)
            all_reward_collect.append(reward_collect)

            print(f" Episode return:{total_reward} ",end='')

        #Discount and normalize rewards
        returns_all = [discount_rewards(x,discount_factor=1) for x in all_reward_collect]
        reward_all = [x for reward_episode in returns_all for x in reward_episode]
        reward_mean = np.mean(reward_all)

        print(f"Mean reward:{reward_mean} ",end='')

        all_reward_collect_discounted = discount_and_normalize_rewards(all_reward_collect,discount_factor)
        return all_obs_collect, all_action_collect, all_reward_collect_discounted

    @property
    def policy(self):
        """ Return the current policy of the agent.

        Returns:
            A `Policy` object
        """

        return self._policy

    @property
    def collect_policy(self):
        """ Return the environment exploration policy of the agent.

        Returns:
            A `Policy` object
        """
        return self._collect_policy





In [ ]:
def test_collected_data(episodes_obs_collect, episodes_action_collect, episodes_reward_collect):
    # Check if all outer lists contain the same number of episodes
    assert len(episodes_obs_collect) == len(episodes_action_collect) == len(episodes_reward_collect), \
        "The number of episodes collected does not match across observations, actions, and rewards."

    # Check if each episode contains the same number of observations, actions, and rewards
    for episode_index in range(len(episodes_obs_collect)):
        num_obs = len(episodes_obs_collect[episode_index])
        num_actions = len(episodes_action_collect[episode_index])
        num_rewards = len(episodes_reward_collect[episode_index])

        assert num_obs == num_actions == num_rewards, \
            f"Episode {episode_index} has mismatched lengths: " \
            f"Observations: {num_obs}, Actions: {num_actions}, Rewards: {num_rewards}"

    print("All tests passed - Data collection is consistent.")

In [ ]:
agent1 = REINFORCEAgent()
# optimizer = torch.optim.Adam(lr=0.001,eps=1e-8,params=agent1.policy.net.parameters())

optimizer = torch.optim.SGD(lr=0.0001,params=agent1.policy.net.parameters())

In [ ]:
# Test agent's `collect`
all_obs_collect, all_action_collect, all_reward_collect_discounted = agent1.collect(env,10)

print(type(all_obs_collect))
print(type(all_action_collect))
print(type(all_reward_collect_discounted))

print(type(all_obs_collect[0]))
print(type(all_action_collect[0]))
print(type(all_reward_collect_discounted[0]))

test_collected_data(all_obs_collect,all_action_collect,all_reward_collect_discounted)

print(len(all_obs_collect))
print(len(all_obs_collect[0]))
print(all_obs_collect[0])

print(len(all_action_collect))
print(len(all_action_collect[0]))
print(all_action_collect[0])

print(len(all_reward_collect_discounted))
print(len(all_reward_collect_discounted[0]))

print(all_reward_collect_discounted[0])

 Episode return:18.0  Episode return:10.0  Episode return:9.0  Episode return:11.0  Episode return:15.0  Episode return:11.0  Episode return:15.0  Episode return:19.0  Episode return:30.0  Episode return:22.0 Mean reward:9.69375 <class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>
All tests passed - Data collection is consistent.
10
18
[array([[ 0.0109067 ,  0.04891024,  0.03626722, -0.04919273]],
      dtype=float32), array([[ 0.01188491,  0.24349388,  0.03528336, -0.33021605]],
      dtype=float32), array([[ 0.01675478,  0.43809628,  0.02867904, -0.6115668 ]],
      dtype=float32), array([[ 0.02551671,  0.6328059 ,  0.01644771, -0.8950806 ]],
      dtype=float32), array([[ 0.03817283,  0.82770103, -0.0014539 , -1.1825484 ]],
      dtype=float32), array([[ 0.05472685,  0.632598  , -0.02510487, -0.89032155]],
      dtype=float32), array([[ 0.0673788,  0.8280514, -0.0429113, -1.1907893]], dtype=float32), array([[ 0.08393984,  0.63351095, -0.

In [ ]:
# Test create batches

obs,action,reward = agent1.collect(env,10)

obs_merged = [x for obs_episode in obs for x in obs_episode]
action_merged = [x for action_episode in action for x in action_episode]
reward_merged = [x for reward_episode in reward for x in reward_episode]

batches_obs, batches_action, batches_reward = create_batches(obs_merged,action_merged,reward_merged,batch_size=16)

print(len(batches_obs))
print(len(batches_action))
print(len(batches_reward))

print(type(batches_obs[0]))
print(type(batches_action[0]))
print(type(batches_reward[0]))

print(batches_obs[0].shape)
print(batches_action[0].shape)
print(batches_reward[0].shape)

print(batches_obs[0].device)
print(batches_action[0].device)
print(batches_reward[0].device)


 Episode return:15.0  Episode return:17.0  Episode return:13.0  Episode return:47.0  Episode return:14.0  Episode return:14.0  Episode return:13.0  Episode return:14.0  Episode return:14.0  Episode return:17.0 Mean reward:1.0 12
12
12
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([16, 4])
torch.Size([16])
torch.Size([16])
cuda:0
cuda:0
cuda:0


In [ ]:
batch_size = 128
n_steps = 2000
n_episodes_per_step=2

for i in range(n_steps):
    obs, action, reward  = agent1.collect(env, n_episodes=n_episodes_per_step)
    obs_merged = [x for obs_episode in obs for x in obs_episode]
    action_merged = [x for action_episode in action for x in action_episode]
    reward_merged = [x for reward_episode in reward for x in reward_episode]
    list_batched_obs, list_batched_actions, list_batched_rewards = create_batches(obs_merged,action_merged,reward_merged,batch_size)

    # print(reward_merged)
    # average_reward = np.mean(reward_merged)

    n_batches = len(list_batched_obs)
    print(f' i:{i}, n batches:{n_batches} ')
    while len(list_batched_obs) != 0:
        batched_obs = list_batched_obs.pop()
        batched_action = list_batched_actions.pop()
        batched_reward = list_batched_rewards.pop()

        batched_action = batched_action.reshape(-1)

        # if j == 0:
        #     print(f'batch_obs shape:{batched_obs.shape}')
        #     print(f'batch_action shape:{batched_action.shape}')
        #     print(f'batch_reward shape:{batched_reward.shape}')
        loss = agent1.train(optimizer,batched_obs,batched_action,batched_reward)

        if i%100 == 0:
            print(f'loss:{loss.item()}')

 Episode return:14.0  Episode return:20.0 Mean reward:9.264705882352942  i:0, n batches:1 
loss:0.015016906894743443
 Episode return:18.0  Episode return:10.0 Mean reward:8.071428571428571  i:1, n batches:1 
 Episode return:19.0  Episode return:20.0 Mean reward:10.256410256410257  i:2, n batches:1 
 Episode return:25.0  Episode return:19.0 Mean reward:11.704545454545455  i:3, n batches:1 
 Episode return:12.0  Episode return:22.0 Mean reward:9.735294117647058  i:4, n batches:1 
 Episode return:19.0  Episode return:23.0 Mean reward:11.095238095238095  i:5, n batches:1 
 Episode return:44.0  Episode return:13.0 Mean reward:18.964912280701753  i:6, n batches:1 
 Episode return:14.0  Episode return:16.0 Mean reward:8.033333333333333  i:7, n batches:1 
 Episode return:9.0  Episode return:21.0 Mean reward:9.2  i:8, n batches:1 
 Episode return:13.0  Episode return:24.0 Mean reward:10.567567567567568  i:9, n batches:1 
 Episode return:11.0  Episode return:11.0 Mean reward:6.0  i:10, n batches

KeyboardInterrupt: 

In [ ]:
print(obs[0])
print(reward[0])

[array([[0.00370961, 0.02219852, 0.02089172, 0.04088289]], dtype=float32), array([[ 0.00415358, -0.17321669,  0.02170938,  0.34008345]],
      dtype=float32), array([[ 0.00068924, -0.3686407 ,  0.02851105,  0.63953245]],
      dtype=float32), array([[-0.00668357, -0.17392762,  0.0413017 ,  0.35596263]],
      dtype=float32), array([[-0.01016212, -0.3696117 ,  0.04842095,  0.6613777 ]],
      dtype=float32), array([[-0.01755436, -0.56537277,  0.0616485 ,  0.9689053 ]],
      dtype=float32), array([[-0.02886181, -0.37113026,  0.08102661,  0.69620824]],
      dtype=float32), array([[-0.03628442, -0.17721994,  0.09495077,  0.4300928 ]],
      dtype=float32), array([[-0.03982881, -0.37354922,  0.10355263,  0.7511338 ]],
      dtype=float32), array([[-0.0472998 , -0.17999619,  0.1185753 ,  0.49274957]],
      dtype=float32), array([[-0.05089973,  0.01327123,  0.12843029,  0.23966359]],
      dtype=float32), array([[-0.0506343 , -0.18342914,  0.13322356,  0.56993896]],
      dtype=float32), a

In [ ]:
print(obs[0][24])
print(action[0][24])
print(reward[0][24])

IndexError: list index out of range

In [ ]:
def get_frames_single_episode(env,agent):
    frames = []
    total_reward_raw = 0
    t = 0

    obs, info = env.reset()
    obs = obs.reshape(1,-1)
    obs_tensor = torch.from_numpy(obs).cuda()
    reward = np.asarray(0, dtype= np.float32)
    step_type = np.asarray(0, dtype= np.float32) # step type with content value of 0 means a FIRST step type
    discount_factor = 1

    time_step = TimeStep(obs_tensor,reward,step_type,discount_factor)
    total_reward_raw += reward

    frames.append(env.render())

    done, truncate = False, False
    while not (done or truncate):
        policy_step = agent.policy.action(time_step)
        action_int = policy_step.action.reshape(-1).item() # has to convert the tensor to int since the environment accepts the action as int
        action_int = int(action_int)
        # print(f"step:{t},total_reward:{total_reward_raw},action:{action_int}")

        obs, reward, done, truncate, info = env.step(action_int)
        obs = obs.reshape(1,-1)
        obs_tensor = torch.from_numpy(obs).cuda()

        if done or truncate:
            step_type = np.asarray(2,dtype=np.int32) # step type with content value of 2 means a LAST step type
        else:
            step_type = np.asarray(1,dtype=np.int32) # step type with content value of 1 means a MID step type

        time_step = TimeStep(obs_tensor,reward,step_type,discount_factor)
        frames.append(env.render())
        total_reward_raw += reward

    print(f"total reward:{total_reward_raw}")
    print(f"last reward:{reward}")

    return frames

In [ ]:
frames = get_frames_single_episode(env=env,agent=agent1)

total reward:254.0
last reward:1.0


In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import matplotlib

# extra code – this cell displays an animation of one episode

def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = matplotlib.animation.FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim


anime = plot_animation(frames)


In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

anime.save('trial5.mp4', writer=writer)